<a href="https://colab.research.google.com/github/bipinr8/MH_cash_deployment/blob/main/Morgan_Housel_Cash_deployment_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Morgan Housel's cash deployment strategy**.

How to deploy surplus cash, according to Morgan Housel
When the market crashes 10%, deploy


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Importing libraries
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

In [3]:
# reading excel file into data frame
df_nifty100 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/MH/Nifty100.xlsx',
                       sheet_name='Sheet1')

print ('Data read into a pandas dataframe!')

Data read into a pandas dataframe!


In [5]:

# checking data frame
df_nifty100.head()

,Date,Nifty 100
0,2003-01-01,1000.00
1,2003-01-02,1008.03
2,2003-01-03,1004.49
3,2003-01-06,999.85
4,2003-01-07,997.81
